# LangChain 기반 Sub-Agent 컨텍스트 격리 및 Task Delegation 튜토리얼

이 튜토리얼에서는 LangChain 프레임워크를 활용하여 복잡한 에이전트 시스템에서 컨텍스트 격리(context isolation)를 구현하는 방법을 다룹니다. 특히, Sub-agent(서브 에이전트)를 활용하여 각 작업을 독립적으로 처리하고, Supervisor Agent(감독 에이전트)가 Task Delegation(작업 위임) 도구를 통해 효율적으로 업무를 분배하는 구조를 설명합니다. 실습 예제와 함께 Sub-agent의 생성, 도구 할당, 그리고 실제 연구 에이전트 구축 과정을 단계별로 안내합니다.

**🏆 주로 다루는 내용**
1. 컨텍스트 격리의 개념과 Sub-agent의 역할  
   - 에이전트 시스템에서 컨텍스트 충돌을 방지하기 위한 격리 전략과 Sub-agent의 필요성 및 동작 원리 소개
2. Sub-agent 생성 및 구성 방법  
   - `SubAgent` 타입 정의와 각 Sub-agent의 이름, 설명, 프롬프트, 도구 할당 방식 구현
3. Task Delegation 도구 개발  
   - Supervisor Agent가 Sub-agent에게 작업을 위임하는 `task` 도구의 구조와 동작 방식 설명
4. Sub-agent를 활용한 연구 시스템 예제  
   - Supervisor Agent와 Research Sub-agent를 활용하여 실제로 정보를 검색하고 결과를 반환하는 예제 구현
5. LangChain 1.0 버전의 주요 변경 사항  
   - `create_react_agent`가 `create_agent`로 변경된 점과 코드 내 적용 방법 안내

**✅ 주요 키워드**
- Sub-agent
- Context Isolation
- Task Delegation
- LangChain
- Agent Registry
- ToolMessage
- Supervisor Agent
- TypedDict
- create_agent

**⭐️ 사전 지식**
- Python 기본 문법 및 클래스 사용법
- LangChain 프레임워크의 기본 구조 및 에이전트 개념
- 함수형 프로그래밍 및 타입 힌트(typing, TypedDict)
- Jupyter Notebook 사용 경험
- AI 에이전트 시스템의 기본 원리(권장)

In [ ]:
from dotenv import load_dotenv
from langchain_teddynote import logging

# 환경 변수 로드
load_dotenv(override=True)
# 추적을 위한 프로젝트 이름 설정
logging.langsmith("Deep-Agent-Tutorial")

## 컨텍스트 격리: Sub-agent 활용

<img src="./assets/agent_header_subagent.png" width="800" style="display:block; margin-left:0;">

복잡한 에이전트 시스템에서는 대화가 진행될수록 에이전트의 컨텍스트가 빠르게 증가하여 다양한 문제를 야기할 수 있습니다. 대표적인 문제는 컨텍스트 충돌이나 혼란으로, 여러 목적이 하나의 컨텍스트 윈도우에 혼합될 경우 에이전트의 성능이 저하될 수 있습니다. 이러한 문제를 해결하기 위해 **컨텍스트 격리(context isolation)** 전략이 활용됩니다.  
컨텍스트 격리는 각 작업을 독립적으로 처리할 수 있는 **특화된 Sub-agent**에게 업무를 분배함으로써, 각 Sub-agent가 자신만의 격리된 컨텍스트 윈도우에서 동작하도록 합니다. 이를 통해 컨텍스트 충돌, 혼란, 오염, 희석 현상을 방지하고, 각 작업에 집중할 수 있는 환경을 제공합니다.  
자세한 내용은 [LangChain 공식 블로그의 컨텍스트 엔지니어링](https://blog.langchain.com/context-engineering-for-agents/)과 [Anthropic의 멀티 에이전트 연구 시스템](https://www.anthropic.com/engineering/multi-agent-research-system)에서 확인할 수 있습니다.

---

### Sub-agent 위임 구조

![./assets/subagents.png](./assets/subagents.png)

Sub-agent 구조의 핵심은 각 작업에 맞는 도구(tool set)를 가진 Sub-agent를 생성하여, 업무를 효율적으로 분리하는 데 있습니다. 각 Sub-agent는 `subagent_type`을 키로 하는 레지스트리 딕셔너리(registry dictionary)에 저장되며, 메인 에이전트는 `task(description, subagent_type)` 형태의 도구 호출을 통해 작업을 위임합니다.  
Sub-agent는 부모 에이전트의 컨텍스트와 완전히 분리된 환경에서 동작하며, 작업 결과는 `ToolMessage` 형태로 부모 에이전트에게 반환됩니다. 이를 통해 역할과 책임이 명확하게 분리되어, 시스템의 안정성과 확장성을 높일 수 있습니다.

## Step 1: Sub-agent 생성

복잡한 에이전트 시스템에서 각 작업을 독립적으로 처리하기 위해서는, 먼저 **Sub-agent**를 정의해야 합니다. 이를 위해 `TypedDict`를 활용하여 각 Sub-agent의 구성 요소를 명확하게 지정할 수 있습니다.

```python
from typing_extensions import TypedDict

class SubAgent(TypedDict):
    """Configuration for a specialized sub-agent."""

    name: str
    description: str
    prompt: str
    tools: NotRequired[list[str]]
```

위와 같이 `SubAgent` 타입을 정의하면, 각 Sub-agent의 이름, 설명, 프롬프트, 그리고 사용할 도구 목록을 체계적으로 관리할 수 있습니다. 이후, 여러 개의 Sub-agent를 리스트 형태로 생성하여, 시스템에서 접근 가능한 모든 Sub-agent를 등록할 수 있습니다.

```python
agents: list[SubAgent] = ...
subagents = {
    agent['name']: create_react_agent(
        model=model,
        prompt=agent['prompt'],
        tools=get_tools(agent['tools']),
        ...
    )
}
```

이 구조를 통해 각 Sub-agent는 자신만의 역할과 도구를 갖추고, 독립적으로 동작할 수 있습니다.

---

## Step 2: Sub-agent를 활용한 Task Delegation 도구 개발

Sub-agent에게 특정 작업을 위임하기 위해서는, 작업을 전달하고 결과를 받아오는 **Task Delegation** 도구가 필요합니다. 아래 예시와 같이, `task` 함수는 작업 설명과 사용할 Sub-agent 타입을 입력받아, 해당 Sub-agent에게 업무를 위임합니다.

```python
def task(
    description: str,  # Sub-agent가 수행할 작업 설명
    subagent_type: str  # 사용할 Sub-agent 타입
):
    # 작업 설명만 포함된 새로운 메시지 생성
    # Sub-agent 호출
    # Sub-agent의 응답 및 파일 시스템 변경 사항을 상태에 반영
```

실제 구현에서는 다음과 같이 동작합니다.

```python
@tool(description=TASK_DESCRIPTION_PREFIX.format(other_agents=other_agents_string))
def task(
    description: str,
    subagent_type: str,
    state: Annotated[DeepAgentState, InjectedState],
    tool_call_id: Annotated[str, InjectedToolCallId],
):
    """Delegate a task to a specialized sub-agent with isolated context.

    This creates a fresh context for the sub-agent containing only the task description,
    preventing context pollution from the parent agent's conversation history.
    """
    # 요청된 Sub-agent 타입이 존재하는지 검증
    if subagent_type not in agents:
        return f"Error: invoked agent of type {subagent_type}, the only allowed types are {[f'`{k}`' for k in agents]}"

    # 해당 Sub-agent 가져오기
    sub_agent = agents[subagent_type]

    # 작업 설명만 포함된 격리된 컨텍스트 생성 (부모 에이전트의 히스토리 미포함)
    state["messages"] = [{"role": "user", "content": description}]

    # Sub-agent를 격리된 환경에서 실행
    result = sub_agent.invoke(state)

    # 작업 결과를 부모 에이전트에게 ToolMessage 형태로 반환
    return Command(
        update={
            "files": result.get("files", {}),  # 파일 변경 사항 병합
            "messages": [
                ToolMessage(
                    result["messages"][-1].content, tool_call_id=tool_call_id
                )
            ],
        }
    )

return task
```

이와 같은 방식으로, 각 Sub-agent는 독립된 컨텍스트에서 작업을 수행하며, 부모 에이전트의 대화 기록과 분리되어 컨텍스트 오염을 방지할 수 있습니다. 결과적으로, 시스템의 안정성과 확장성이 크게 향상됩니다.

자세한 컨텍스트 엔지니어링 및 멀티 에이전트 시스템 구현 방법은 [LangChain 공식 블로그](https://blog.langchain.com/context-engineering-for-agents/)와 [Anthropic의 멀티 에이전트 연구 시스템](https://www.anthropic.com/engineering/multi-agent-research-system)에서 확인할 수 있습니다.

In [ ]:
%%writefile ../src/deep_agents_from_scratch/task_tool.py
"""Task delegation tools for context isolation through sub-agents.

This module provides the core infrastructure for creating and managing sub-agents
with isolated contexts. Sub-agents prevent context clash by operating with clean
context windows containing only their specific task description.
"""

from typing import Annotated, NotRequired
from typing_extensions import TypedDict

from langchain_core.messages import ToolMessage
from langchain_core.tools import BaseTool, InjectedToolCallId, tool
from langgraph.prebuilt import InjectedState  # updated 1.0
from langchain_core.agents import create_agent  # updated 1.0

from langgraph.types import Command

from deep_agents_from_scratch.prompts import TASK_DESCRIPTION_PREFIX
from deep_agents_from_scratch.state import DeepAgentState


class SubAgent(TypedDict):
    """Configuration for a specialized sub-agent."""

    name: str
    description: str
    prompt: str
    tools: NotRequired[list[str]]


def _create_task_tool(tools, subagents: list[SubAgent], model, state_schema):
    """Create a task delegation tool that enables context isolation through sub-agents.

    This function implements the core pattern for spawning specialized sub-agents with
    isolated contexts, preventing context clash and confusion in complex multi-step tasks.

    Args:
        tools: List of available tools that can be assigned to sub-agents
        subagents: List of specialized sub-agent configurations
        model: The language model to use for all agents
        state_schema: The state schema (typically DeepAgentState)

    Returns:
        A 'task' tool that can delegate work to specialized sub-agents
    """
    # Sub-agent 레지스트리 딕셔너리 생성, 이름을 키로 하여 에이전트 인스턴스 저장
    agents = {}

    # 도구 이름별 매핑 딕셔너리 생성, Sub-agent별 도구 할당에 활용
    tools_by_name = {}
    for tool_ in tools:
        if not isinstance(tool_, BaseTool):
            tool_ = tool(tool_)
        tools_by_name[tool_.name] = tool_

    # Sub-agent 구성 정보 기반으로 특화 에이전트 생성 및 레지스트리에 등록
    for _agent in subagents:
        if "tools" in _agent:
            # Sub-agent에 지정된 도구만 할당
            _tools = [tools_by_name[t] for t in _agent["tools"]]
        else:
            # 도구 미지정 시 전체 도구 할당
            _tools = tools
        agents[_agent["name"]] = create_agent(   # updated 1.0
            model, system_prompt=_agent["prompt"], tools=_tools, state_schema=state_schema
        )

    # 사용 가능한 Sub-agent 목록을 도구 설명에 활용하기 위한 문자열 리스트 생성
    other_agents_string = [
        f"- {_agent['name']}: {_agent['description']}" for _agent in subagents
    ]

    @tool(description=TASK_DESCRIPTION_PREFIX.format(other_agents=other_agents_string))
    def task(
        description: str,
        subagent_type: str,
        state: Annotated[DeepAgentState, InjectedState],
        tool_call_id: Annotated[str, InjectedToolCallId],
    ):
        """Delegate a task to a specialized sub-agent with isolated context.

        This creates a fresh context for the sub-agent containing only the task description,
        preventing context pollution from the parent agent's conversation history.
        """
        # 요청된 Sub-agent 타입이 레지스트리에 존재하는지 검증, 미존재 시 에러 반환
        if subagent_type not in agents:
            return f"Error: invoked agent of type {subagent_type}, the only allowed types are {[f'`{k}`' for k in agents]}"

        # 요청된 Sub-agent 인스턴스 가져오기
        sub_agent = agents[subagent_type]

        # 작업 설명만 포함된 격리된 컨텍스트 생성, 부모 에이전트의 히스토리 미포함
        state["messages"] = [{"role": "user", "content": description}]

        # 격리된 환경에서 Sub-agent 실행 및 결과 획득
        result = sub_agent.invoke(state)

        # 작업 결과를 Command 객체로 래핑하여 부모 에이전트에 ToolMessage 형태로 반환
        return Command(
            update={
                "files": result.get("files", {}),  # 파일 변경 사항 병합
                "messages": [
                    # Sub-agent의 마지막 메시지를 ToolMessage로 변환하여 반환
                    ToolMessage(
                        result["messages"][-1].content, tool_call_id=tool_call_id
                    )
                ],
            }
        )

    return task


이제 Sub-agent를 도구로 생성하는 루틴이 준비되었습니다. 구체적인 Sub-agent를 정의하고, 시스템이 `task` 도구를 통해 이들을 호출할 수 있도록 설정할 수 있습니다.  
앞서 설명한 `_create_task_tool` 함수는 `SubAgent` 타입의 리스트를 입력받으며, 이 리스트에는 생성할 각 에이전트의 설명이 포함되어 있습니다.

```python
class SubAgent(TypedDict):
    """Configuration for a specialized sub-agent."""

    name: str
    description: str
    prompt: str
    tools: NotRequired[list[str]]


def _create_task_tool(tools, subagents: list[SubAgent], model, state_schema):
    """Create a task delegation tool that enables context isolation through sub-agents.
```

`SubAgent` 클래스는 Sub-agent가 도구이자 에이전트로서의 역할을 모두 수행할 수 있도록 필요한 정보를 정의합니다.

- **도구로서의 역할**: Sub-agent는 Supervisor Agent에게 자신의 기능과 호출 방법에 대한 정보를 제공합니다.
- **에이전트로서의 역할**: 각 Sub-agent는 자신이 수행할 작업 방식에 대한 프롬프트와, 해당 작업에 특화된 도구 세트를 필요로 합니다.

아래 예시는 연구용 Sub-agent를 생성하는 코드입니다.  
`description`에는 Supervisor Agent가 이 Sub-agent에게 단일 주제의 연구 작업만 위임해야 함을 명시합니다.  
`SIMPLE_RESEARCH_INSTRUCTIONS`는 Sub-agent가 연구를 수행할 때 사용할 프롬프트로, 예시에서는 간단하게 작성되어 있지만, 범용 연구 에이전트의 경우 더 상세하게 구성할 수 있습니다.  
또한, Sub-agent에는 연구 수행을 위한 `web_search` 도구가 할당됩니다.

```python
# Create research sub-agent
research_sub_agent = {
    "name": "research-agent",
    "description": "Delegate research to the sub-agent researcher. Only give this researcher one topic at a time.",
    "prompt": SIMPLE_RESEARCH_INSTRUCTIONS,
    "tools": ["web_search"],
}
```

이처럼 Sub-agent는 특정 작업과 해당 작업에 필요한 도구를 함께 전달받아, 오직 해당 작업 설명에 한정된 독립적인 컨텍스트에서 동작합니다.  
이러한 **컨텍스트 엔지니어링**([LangChain 공식 블로그](https://blog.langchain.com/context-engineering-for-agents/)) 전략을 통해 Sub-agent의 작업 환경은 컨텍스트 충돌, 혼란, 오염, 희석 현상으로부터 안전하게 보호됩니다.

Supervisor Agent의 프롬프트에는 이제 Sub-agent를 호출하고 활용하는 방법에 대한 설명이 반드시 포함되어야 합니다.  
아래 예시에서는 *Available Tools* 항목과, 필요 시 병렬 연구를 활용하도록 안내하는 지침이 포함되어 있습니다.  
이와 같이 Sub-agent 구조를 명확히 정의하고, Supervisor Agent가 이를 적절히 활용할 수 있도록 프롬프트를 설계하는 것이 중요합니다.

In [ ]:
# Sub-agent 사용법 프롬프트 출력 함수 호출, Supervisor Agent 프롬프트 설계 참고 목적
from utils import show_prompt

from deep_agents_from_scratch.prompts import SUBAGENT_USAGE_INSTRUCTIONS

show_prompt(SUBAGENT_USAGE_INSTRUCTIONS)

이제 Supervisor Agent와 Sub-agent를 활용한 연구 시스템의 기본 구조를 구현해 보겠습니다. 이번 단계에서는 실제 웹 검색 결과 대신 미리 정의된 검색 결과를 사용하여 전체 시스템의 동작 방식을 간단하게 시연합니다. 이를 통해 Sub-agent와 Supervisor Agent가 어떻게 협력하여 정보를 처리하는지 구조를 이해할 수 있습니다. 다음 강의에서는 실제 웹 검색 기능을 포함한 완성형 연구 시스템을 직접 구축하게 됩니다.

In [ ]:
from datetime import datetime

from IPython.display import Image, display
from langchain.chat_models import init_chat_model
from langchain_core.tools import tool

# from langgraph.prebuilt import create_react_agent
from langchain.agents import create_agent  # 1.0 버전에서 create_agent로 변경됨

from deep_agents_from_scratch.prompts import SUBAGENT_USAGE_INSTRUCTIONS
from deep_agents_from_scratch.state import DeepAgentState
from deep_agents_from_scratch.task_tool import _create_task_tool

# 동시 연구 작업 단위 최대 개수 제한 설정
max_concurrent_research_units = 3
# 연구자 에이전트 반복 횟수 최대값 설정
max_researcher_iterations = 3

# 웹 검색 결과 모킹 데이터 정의, 실제 검색 대신 고정된 결과 반환
search_result = """The Model Context Protocol (MCP) is an open standard protocol developed
by Anthropic to enable seamless integration between AI models and external systems like
tools, databases, and other services. It acts as a standardized communication layer,
allowing AI models to access and utilize data from various sources in a consistent and
efficient manner. Essentially, MCP simplifies the process of connecting AI assistants
to external services by providing a unified language for data exchange. """


# 웹 검색 도구 모킹 함수 정의, 실제 검색 대신 고정된 결과 반환
@tool(parse_docstring=True)
def web_search(
    query: str,
):
    """Search the web for information on a specific topic.

    This tool performs web searches and returns relevant results
    for the given query. Use this when you need to gather information from
    the internet about any topic.

    Args:
        query: The search query string. Be specific and clear about what
               information you're looking for.

    Returns:
        Search results from the search engine.

    Example:
        web_search("machine learning applications in healthcare")
    """
    return search_result


# 연구용 Sub-agent 프롬프트 문자열 정의, 단일 웹 검색만 허용
SIMPLE_RESEARCH_INSTRUCTIONS = """You are a researcher. Research the topic provided to you. IMPORTANT: Just make a single call to the web_search tool and use the result provided by the tool to answer the provided topic."""

# 연구용 Sub-agent 구성 정보 딕셔너리 생성, 이름/설명/프롬프트/도구 지정
research_sub_agent = {
    "name": "research-agent",
    "description": "Delegate research to the sub-agent researcher. Only give this researcher one topic at a time.",
    "prompt": SIMPLE_RESEARCH_INSTRUCTIONS,
    "tools": ["web_search"],
}

# 위에서 초기화된 기본 LLM 사용, 모델 및 온도 설정
model = init_chat_model(model="anthropic:claude-sonnet-4-20250514", temperature=0.0)

# Sub-agent에 할당할 도구 리스트 정의, 웹 검색 도구 포함
sub_agent_tools = [web_search]

# Sub-agent에게 작업 위임을 위한 task 도구 생성, 컨텍스트 격리 구현
task_tool = _create_task_tool(
    sub_agent_tools, [research_sub_agent], model, DeepAgentState
)

# Supervisor Agent가 사용할 도구 리스트, task 도구 포함
delegation_tools = [task_tool]

# Supervisor Agent 생성, 시스템 프롬프트 및 상태 스키마 지정
agent = create_agent(
    model,
    delegation_tools,
    system_prompt=SUBAGENT_USAGE_INSTRUCTIONS.format(
        max_concurrent_research_units=max_concurrent_research_units,
        max_researcher_iterations=max_researcher_iterations,
        date=datetime.now().strftime("%a %b %-d, %Y"),
    ),
    state_schema=DeepAgentState,
)

# 에이전트의 내부 구조 시각화, Mermaid 그래프 이미지 출력
display(Image(agent.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
# 메시지 포맷팅 함수 임포트, 에이전트 응답 메시지 가독성 향상 목적
from utils import format_messages

# 에이전트에게 사용자 질문 전달 및 MCP 개요 요청
result = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "Give me an overview of Model Context Protocol (MCP).",
            }
        ],
    }
)

# 에이전트 응답 메시지 리스트를 보기 좋게 포맷팅하여 출력
format_messages(result["messages"])

아래는 LangChain Sub-agent 컨텍스트 격리 및 Task Delegation 실습의 실행 결과를 확인할 수 있는 트레이스 링크입니다.  
실제 에이전트의 동작 흐름과 메시지 교환 과정을 시각적으로 분석하고 싶다면, 아래 URL을 참고하시기 바랍니다.

- [실행 트레이스 확인하기](https://smith.langchain.com/public/26cc1c2b-e785-4c6d-a2a7-c30a31875fc7/r)
<!-- 이전 트레이스: https://smith.langchain.com/public/edc4e672-db9c-457a-953d-f62e7813591c/r -->

이 링크에서는 Supervisor Agent가 Sub-agent에게 작업을 위임하고, Sub-agent가 독립된 컨텍스트에서 작업을 수행한 후 결과를 반환하는 전체 과정을 단계별로 확인할 수 있습니다.  
실제 시스템의 동작 방식과 컨텍스트 격리 전략이 어떻게 적용되는지 이해하는 데 도움이 됩니다.  
추가적으로, LangChain의 다양한 실습 예제와 트레이스 분석 방법은 공식 문서 및 블로그를 참고하시기 바랍니다.